## Day 7: The Sum of Its Parts

https://adventofcode.com/2018/day/7

### Part 1

Put the alphabetically first step that has no dependencies at the start of the sequence. Repeat until finished. Let `networkx` do the hefty lifting. (This needs version 2+ of `networkx`.)

In [1]:
from parse import parse
import networkx as nx


# instructions are tuples (x, y) where x must be
# sequenced before y
def parse_instructions(strings):
    return [parse('Step {} must be finished before step {} can begin.', s).fixed 
            for s in strings]


def sequence_steps(instructions):
    dependencies = nx.DiGraph()
    dependencies.add_edges_from(instructions)
    
    sequence = ''
    
    while dependencies:
        next_step = min([n for n, d in dependencies.in_degree if d == 0])
        sequence += next_step
        dependencies.remove_node(next_step)
    
    return sequence

In [2]:
test_input = '''Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.'''.splitlines()

test_instructions = parse_instructions(test_input)
test_instructions

[('C', 'A'),
 ('C', 'F'),
 ('A', 'B'),
 ('A', 'D'),
 ('B', 'E'),
 ('D', 'E'),
 ('F', 'E')]

In [3]:
sequence_steps(test_instructions)

'CABDFE'

In [4]:
instructions = parse_instructions(open('input', 'r'))

sequence_steps(instructions)

'BCADPVTJFZNRWXHEKSQLUYGMIO'

### Part 2

This is possibly the first problem where Microsoft Project is the best tool for the job.

In [5]:
import itertools


def project_estimate(instructions, n_workers, step_duration_minimum = 60):
    
    def task_length(task):
        return ord(task) - ord('A') + step_duration_minimum + 1

    dependencies = nx.DiGraph()
    dependencies.add_edges_from(instructions)
    
    # Dictionary of tasks with time left to complete
    running_tasks = {}
    
    for time_taken in itertools.count():
    
        # Remove finished tasks from dependencies and 
        # free the workers 
        finished_tasks = [t for t in running_tasks 
                          if running_tasks[t] == 0]
        for task in finished_tasks:
            dependencies.remove_node(task)
            del running_tasks[task]
            
        # What tasks have no further dependencies?    
        open_tasks = [n for n, d in dependencies.in_degree 
                      if d == 0 and n not in running_tasks]
        
        # If there's nothing more to be done then the time 
        # taken is the result
        if not open_tasks and not running_tasks:
            return time_taken
        
        # Is there anything to do and anyone free to do it?
        while open_tasks and len(running_tasks) < n_workers:
            next_task = open_tasks.pop()
            running_tasks[next_task] = task_length(next_task)
            
        for task in running_tasks:
            running_tasks[task] -= 1
    

In [6]:
assert project_estimate(test_instructions, 2, 0) == 15

In [7]:
project_estimate(instructions, 5)

973